In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -q /content/drive/MyDrive/face_age.zip

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU is set and ready!")
    except RuntimeError as e:

        print(e)

In [ ]:

import numpy as np
import os
import pandas as pd
import kagglehub
import tensorflow as tf
import cv2
import imghdr
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, Accuracy
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Add, UpSampling2D, Layer
from tensorflow.keras.layers import Activation, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import load_model, Model

In [ ]:
tf.random.set_seed(42)

In [ ]:
data_dir = 'face_age'

In [ ]:
image_exts = ["jpeg","jpg","bmp","png"]
for image_class in os.listdir(data_dir):
    class_path = os.path.join(data_dir, image_class)
    if not os.path.isdir(class_path):
        continue

    for image in os.listdir(class_path):
        image_path = os.path.join(class_path, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)

            if tip not in image_exts:
                print(f"Not a valid image: {image_path}")
                os.remove(image_path)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

In [ ]:
import glob
for ds_store_file in glob.glob(f"{data_dir}/**/.DS_Store", recursive=True):
    os.remove(ds_store_file)

In [ ]:

class_names = sorted(os.listdir(data_dir))
age_labels = [int(name) for name in class_names]  # Convert subdir names to integers

# lookup table for class index → actual age
age_lookup = tf.constant(age_labels, dtype=tf.int32)

# 3. Map dataset labels to actual ages
data = tf.keras.utils.image_dataset_from_directory(data_dir)
data = data.map(lambda x, y: (x, tf.gather(age_lookup, y)))

# 4. Now apply your age-to-category mapping
def label_to_category(image, label):
    category = tf.where(
        label < 13, 0,
        tf.where(
            label < 20, 1,
            tf.where(label < 60, 2, 3)
        )
    )
    return image, tf.cast(category, tf.int32)

dataset = data.map(label_to_category)


In [ ]:
dataset = dataset.map(lambda x,y:(x/255,y))

In [ ]:
scaled_iterator_1 = dataset.as_numpy_iterator()

In [ ]:
batch_dataset = scaled_iterator_1.next()

In [ ]:
length = dataset.cardinality().numpy()

train_size_d = int(length * 0.7)
val_size_d = int(length * 0.2)
test_size_d = length - train_size_d - val_size_d

train_dataset = dataset.take(train_size_d)
val_dataset = dataset.skip(train_size_d).take(val_size_d)
test_dataset = dataset.skip(train_size_d + val_size_d)


### **Block 1 and Block 2  Splitting Training set**

In [ ]:
split_point_1 = train_size_d // 2

# Create the first set of training and validation datasets
block_train_1 = train_dataset.take(split_point_1)
block_val_1 = val_dataset.take(val_size_d // 2)

# Create the second set of training and validation datasets
block_train_2 = train_dataset.skip(split_point_1)
block_val_2 = val_dataset.skip(val_size_d // 2)

print("Block 1 Training dataset size:", block_train_1.cardinality().numpy())
print("Block 1 Validation dataset size:", block_val_1.cardinality().numpy())
print("Block 2 Training dataset size:", block_train_2.cardinality().numpy())
print("Block 2 Validation dataset size:", block_val_2.cardinality().numpy())


In [ ]:
for x, y in block_train_1.take(1):
    print("Training x shape:", x.shape)
    print("Training y shape:", y.shape)

for x, y in block_val_1.take(1):
    print("Validation x shape:", x.shape)
    print("Validation y shape:", y.shape)

In [ ]:
mobilenet_train_1 = block_train_1
mobilenet_val_1 = block_val_1


In [ ]:
for x, y in mobilenet_train_1.take(1):
    print("Training x shape:", x.shape)
    print("Training y shape:", y.shape)

for x, y in block_val_1.take(1):
    print("Validation x shape:", x.shape)
    print("Validation y shape:", y.shape)

# **Part 1  Auto encoder**

In [ ]:
input_img = Input(shape=(256, 256, 3))

In [ ]:

# encodes the image
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

#  Decoder(it tries to re- construct the encoded image)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# autoencoder Model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(), loss=MeanSquaredError())

# data
block_train_1 = block_train_1.map(lambda x, y: (x, x))
block_val_1 = block_val_1.map(lambda x, y: (x, x))

# training the autoencoder
history = autoencoder.fit(
    block_train_1,
    epochs=30,
    batch_size=64,
    shuffle=True,
    validation_data=block_val_1
)

autoencoder.save("autoencoder_block1_model.h5")

# **Part 2 Build Classification model on the Auto-encoder**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Layer

# Define custom Cast layer
class Cast(Layer):
    def __init__(self, dtype, **kwargs):
        super(Cast, self).__init__(**kwargs)
        self._dtype = dtype

    def call(self, inputs):
        return tf.cast(inputs, self._dtype)

    def get_config(self):
        config = super(Cast, self).get_config()
        config.update({"dtype": self._dtype})
        return config

# Define custom TypeConverter layer
class TypeConverter(Layer):
    def __init__(self, data_type, **kwargs):
        super(TypeConverter, self).__init__(**kwargs)
        self._data_type = data_type

    def call(self, inputs):
        return tf.cast(inputs, self._data_type)

    def get_config(self):
        config = super(TypeConverter, self).get_config()
        config.update({"data_type": self._data_type})
        return config

# Load model with custom layers
autoencoder = load_model(
    "autoencoder_block1_model.h5",
    custom_objects={'TypeConverter': TypeConverter, 'Cast': Cast}
)

# Extract encoder
feature_extractor = Model(
    inputs=autoencoder.input,
    outputs=autoencoder.layers[-2].output
)

# Freeze encoder layers
for layer in feature_extractor.layers:
    layer.trainable = False


In [ ]:
# Classification Model
features = feature_extractor.output
features = GlobalAveragePooling2D()(features)
features = Dense(1024, activation='relu')(features)  # A layer to learn complex patterns
predictions = Dense(4, activation='softmax')(features)  # Softmax gives probabilities
age_classifier = Model(inputs=feature_extractor.input, outputs=predictions)


age_classifier.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
training_history = age_classifier.fit(block_train_2, epochs=10, batch_size=32,validation_data=block_val_2)

In [ ]:

test_loss_1, test_accuracy_1 = age_classifier.evaluate(test_dataset)
print(f"Test Loss: {test_loss_1:.4f}")
print(f"Test Accuracy: {test_accuracy_1:.4f}")

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(training_history.history['loss'], label='Training Loss')
plt.plot(training_history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(training_history.history['accuracy'], label='Training Accuracy')
plt.plot(training_history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


### **Fine tuning the model**




*   Increasing Learning rate
*   Number of nodes to learn complex patterns
*   unfreezing the encoder layers
*   Changed activation to tan-h










In [ ]:
# Freeze encoder layers
for layer in feature_extractor.layers:
    layer.trainable = True

In [ ]:

# Classification Model(fine tuning)
features = Dense(2048, activation='tanh')(features)  # A layer to learn complex patterns
predictions_1 = Dense(4, activation='softmax')(features)  # Softmax gives probabilities
age_classifier_1 = Model(inputs=feature_extractor.input, outputs=predictions_1)


age_classifier_1.compile(
    optimizer=Adam(learning_rate = 0.0003),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
training_history = age_classifier_1.fit(block_train_2, epochs=10, batch_size=32,validation_data=block_val_2)

In [ ]:

test_loss_2, test_accuracy_2 = age_classifier_1.evaluate(test_dataset)
print(f"Test Loss: {test_loss_2:.4f}")
print(f"Test Accuracy: {test_accuracy_2:.4f}")

### **2nd optimization**



*   Increasing Learning rate
*   Changed activation to relu again
*   Increase Epochs
*   using Flatten















In [ ]:

# Classification Model(fine tuning)
features = Flatten()(features)
features = Dense(2048, activation='relu')(features)  # A layer to learn complex patterns
predictions_2 = Dense(4, activation='softmax')(features)  # Softmax gives probabilities
age_classifier_2 = Model(inputs=feature_extractor.input, outputs=predictions_2)


age_classifier_2.compile(
    optimizer=Adam(learning_rate = 0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
training_history = age_classifier_2.fit(block_train_2, epochs=20, batch_size=32,validation_data=block_val_2)

In [ ]:

test_loss_3, test_accuracy_3 = age_classifier_2.evaluate(test_dataset)
print(f"Test Loss: {test_loss_3:.4f}")
print(f"Test Accuracy: {test_accuracy_3:.4f}")

### **Last optimization**


*   Decreasing nodes in layer



In [ ]:
features = Flatten()(features)
features = Dense(64, activation='relu')(features)
features = Dense(128, activation='relu')(features)
features = Dense(264, activation='relu')(features)
features = Dense(1024, activation='relu')(features)
predictions_3 = Dense(4, activation='softmax')(features)  # Softmax gives probabilities
age_classifier_3 = Model(inputs=feature_extractor.input, outputs=predictions_3)

age_classifier_3.compile(
    optimizer=Adam(learning_rate = 0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:

training_history = age_classifier_3.fit(block_train_2, epochs=10, batch_size=32,validation_data=block_val_2)

In [ ]:

test_loss_4, test_accuracy_4 = age_classifier_3.evaluate(test_dataset)
print(f"Test Loss: {test_loss_4:.4f}")
print(f"Test Accuracy: {test_accuracy_4:.4f}")

### **PreTrained Mobile net model**

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load base MobileNetV2 without top layer, accepting 256x256 input
base_model = MobileNetV2(
    input_shape=(256, 256, 3),  # Changed input shape to 256x256
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze initial layers

inputs = Input(shape=(256, 256, 3))  # Input shape changed here as well
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(4, activation='softmax')(x)

mobilenet_model = models.Model(inputs, outputs)

mobilenet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

mobilenet_model.summary()

### **Train**

In [ ]:
mobilenet = mobilenet_model.fit(
    mobilenet_train_1 ,
    validation_data= mobilenet_val_1,
    epochs=20,
)

### **Test**

In [ ]:
test_loss, test_acc = mobilenet_model.evaluate(test_dataset)
print(f"MobileNetV2 Test Accuracy: {test_acc:.2%}")

### **Comparing Model**

In [ ]:
model_names = ['Autoencoder Model 1', 'Autoencoder Model 2', 'Autoencoder Model 3', 'Autoencoder Model 4', 'MobileNetV2']
accuracies = [test_accuracy_1, test_accuracy_2, test_accuracy_3, test_accuracy_4, test_acc]

# Find the model with the highest accuracy
best_model_index = np.argmax(accuracies)
best_model_name = model_names[best_model_index]
best_accuracy = accuracies[best_model_index]

print(f"The best performing model is {best_model_name} with an accuracy of {best_accuracy:.4f}")

# Plotting the accuracy graph
plt.figure(figsize=(10, 6))
plt.plot(model_names, accuracies, marker='o')
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Comparison of Model Accuracies")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()


In [ ]:
from tensorflow import keras

# Save regression model
age_classifier_3.save('age_classifier_3.keras')

# Save classification model
mobilenet_model.save('mobilenet_model.keras')


In [ ]:
import os
import shutil

os.makedirs("part_2_models", exist_ok=True)
shutil.move("age_classifier_3.keras", "part_2_models/age_classifier_3.keras")
shutil.move("mobilenet_model.keras", "part_2_models/mobilenet_model.keras")
shutil.make_archive("part_2_models", 'zip', "part_2_models")
